In [ ]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import matplotlib.pyplot as plt
import numpy as np
import tempfile

import momenta.utils.flux
import momenta.utils.conversions
import momenta.io.parameters
import momenta.io.gw
import momenta.io.transient
import momenta.io.neutrinos
import momenta.io.neutrinos_irfs
import momenta.stats.run
import momenta.stats.constraints

# Inputs

### Parameters and flux

The general parameters of the analysys may be loaded from a YAML file.

In [ ]:
config_str = """
    skymap_resolution: 8
    detector_systematics: 1

    analysis:
        likelihood: poisson
        prior_normalisation:
            variable: flux
            type: flat-linear
            range:
                min: 0
                max: 1.0e+10
"""
paramfile = f"{tempfile.mkdtemp()}/paramfile.yaml"
with open(paramfile, "w") as f:
    f.write(config_str)

parameters = momenta.io.parameters.Parameters(paramfile)

To define the flux, you can use one of the classes inheriting from `momenta.utils.flux.FluxBase`.

In [ ]:
print("Available ready-to-use flux classes:")
for c in momenta.utils.flux.FluxBase.__subclasses__():
    print("\t", c)
    
# example
flux_example_1 = momenta.utils.flux.FluxFixedPowerLaw(emin=0.1, emax=1e6, gamma=2)
print("Example flux: \n\t", flux_example_1)

Another solution is to build your flux function using the different base ingredients inheriting from `momenta.utils.flux.Component`. For each component, a jet structure may be assigned using a class inheriting from `momenta.utils.conversion.JetModelBase`.

In [ ]:
print("Available ready-to-use flux component classes:")
for c in momenta.utils.flux.Component.__subclasses__():
    print("\t", c)
    
print("Available ready-to-use jet model classes:")
for c in momenta.utils.conversions.JetModelBase.__subclasses__():
    print("\t", c)
    
# example
flux_example_2 = momenta.utils.flux.FluxBase()
c1 = momenta.utils.flux.FixedPowerLaw(emin=0.1, emax=1e6, gamma=2)
c1.set_jet(momenta.utils.conversions.JetIsotropic())  # this is the default so has no effect
c2 = momenta.utils.flux.QuasithermalSpectrum(emean=100, alpha=2)
c2.set_jet(momenta.utils.conversions.JetRectangular(np.deg2rad(10)))
flux_example_2.components = [c1, c2]

print("Example flux: \n\t", flux_example_2)

You can also build your own `Component`.

In [ ]:
class MyComponent(momenta.utils.flux.Component):
    
    def __init__(self, emin: float, emax: float, gamma: float, cutoff: float):
        super().__init__(emin=emin, emax=emax, store="exact")
        self.shapefix_names = ["gamma", "cutoff"]
        self.shapefix_values = [gamma, cutoff]
        
    def evaluate(self, energy: np.ndarray):
        return np.where((self.emin <= energy) & (energy <= self.emax), np.power(energy, -self.shapefix_values[0]) * np.exp(energy/self.shapefix_values[1]))

<span style="color:red">Create your own flux object here, using one above (e.g., `flux = flux_example_1`) or creating your own from scratch. This flux is then provided to the parameter object.</span>

In [ ]:
flux = ...
parameters.set_flux(flux)

### Astrophysical source

The easiest option to define the source is `momenta.io.PointSource`, to define a well-defined source with precise direction. An alternative is to use a GW skymap (FITS file) and posterior samples (HDF5 file) as inputs.

In [ ]:
src_example_1 = momenta.io.transient.PointSource(ra_deg=123.45, dec_deg=-42.42, err_deg=1, name="MySrc")
src_example_1.set_distance(124.75)

src_example_2 = momenta.io.gw.GW(
    name="GW190412",
    path_to_fits="input_files/gw_catalogs/GW190412/GW190412_PublicationSamples.fits",
    path_to_samples="input_files/gw_catalogs/GW190412/GW190412_subset.h5",
)
src_example_2.set_parameters(parameters)

<span style="color:red">Create your own source object.</span>

In [ ]:
src = ...

### Neutrino inputs

First, you should define your detector, eventually containing different analysis samples. This can be done using a `momenta.io.neutrinos.NuDetector` object. If you want later to combine different detectors, the relevant object is `momenta.io.neutrinos.SuperNuDetector`, to which individual detectors can be added using the function `add_detector()`.

In [ ]:
# the object can be populating using a dictionary with the proper keys...
det_example_1 = momenta.io.neutrinos.NuDetector(
    {
        "name": "Detector1",
        "samples": ["SampleA"],
    }
)
# ... or providing a YAML file with relevant information
det_example_2 = momenta.io.neutrinos.NuDetector("input_files/detector.yaml")

<span style="color:red">Create your own detector object.</span>

In [ ]:
det = ...

As of now, the `NuDetector` objects are relatively empty. We need to add at least:
- the expected number of background events in the search time window
- the observed number of events in the search time window
- the effective area of the selection

If the analysis is not a simple cut-and-count search (where these three quantities suffice), one may also provide:
- the list of observed events (direction, energy, time...)
- the relevant IRFs for signal and background (point-spread function, energy distribution...)

For effective areas, these are generally defined and computed on sky maps tiled using HealPIX. The resolution of the sky map is characterized by one integer `nside` (the larger is `nside`, the smaller are the pixels). The position on the skymap is uniquely defined by one index (for a given position in the sky and a given `nside`, the index is unique). See https://healpix.sourceforge.io/index.php.

In [ ]:
# The background is simply defined by the corresponding prior
bkg_example_1 = momenta.io.neutrinos.BackgroundFixed(0.51)  # expected background is fixed
bkg_example_2 = momenta.io.neutrinos.BackgroundGaussian(0.51, 0.12)  # we include some uncertainty
bkg_example_3 = momenta.io.neutrinos.BackgroundPoisson(Noff=51, alpha_offon=100.)  # background derived from an OFF measurement (bkg = Noff / alpha_offon)

In [ ]:
# The number of observed event is just one integer
obs_example = 0

In [ ]:
# These two quantities are fed to the detector object (as list with one entry per sample)
det_example_1.set_observations(nobserved=[obs_example], background=[bkg_example_1])

In [ ]:
# The effective area may be built from one of the existing classes or another subclass.
print("Available ready-to-use effective area classes:")
for c in momenta.io.neutrinos_irfs.EffectiveAreaBase.__subclasses__():
    print("\t", c)
    
aeff_example = momenta.io.neutrinos_irfs.EffectiveAreaAllSky(func = lambda x: (x/100)**3 * np.exp(-x/1e4))

In [ ]:
# From these, one can easily check the effective area, the number of expected signal events given a flux component...
x = np.logspace(-1, 6, 701)
# The function `evaluate` takes three arguments: the energy, the pixel index, and the `nside` of the corresponding map.
# Here, as we defined an effective area that does not depend on the position, the two latter are not used.
y = aeff_example.evaluate(x, ipix=0, nside=4)
plt.loglog(x, y)
plt.xlabel("Energy [GeV]")
plt.ylabel("Effective area [cm²]")

print(f"Expected number of signal events for a given flux (normalization set to 1): {aeff_example.get_acceptance(c1, 0, 4):.2f}")

In [ ]:
# The effective area object is given to the NuDetector object (as list with one effective area per sample)
det_example_1.set_effective_areas(aeff_example)

<span style="color:red">Populate your detector object with the relevant information.</span>

In [ ]:
# define background expectation
...
# define observed number of events
...
# define effective areas
...

#det.set_observations(..., ...)
#det.set_effective_areas(...)

# Computing constraints

We are ready to run the analysis. This is done by simply calling `momenta.stats.run.run_ultranest`. In the simplest cases, this generally takes a few seconds. For more complexe scenarii, the code may run for a few minutes.

<span style="color:red">Run the following lines as soon as you are ready with all the ingredients</span>

In [ ]:
model, result = momenta.stats.run.run_ultranest(
    detector=det,
    src=src,
    parameters=parameters
)

The first return value, `model`, contains the `momenta.stats.model.ModelOneSource` object that has been used to run the ultranest algorithm. It may be inspect for checks.

In [ ]:
print("Number of dimensions in the likelihood:", model.ndims)
print("List of parameters:", model.param_names)

The second return value, `result`, contains the analysis results, notably in terms of the posterior samples that may be used to derive limits / median...

In [ ]:
print("result =", result.keys(), '\n')

print("samples =", result["samples"].keys())

The main columns are:
- `itoy` is the index of the source characteristics marginalisation. It is usually not useful and kept only for debugging purposes.
- `fluxnorm{i}` is the normalization of the (i+1)th flux component (e.g., `fluxnorm0` = first component), provided in GeV⁻¹ cm⁻².
- `etot{i}` is the total energy emitted in neutrinos by the source from the (i+1)th flux component, provided in erg.
- `fnu{i}` is the ratio between the total energy emitted in neutrinos from the (i+1)th flux component and another reference energy. As of now, this is only implemented for GW sources where the estimated mass difference between the initial and the final systems. extracted from LVK posterior samples, is used.
- `etot` is the total energy emitted in neutrinos, summing over all components (in erg).
- `fnu` is the ratio between the total energy emitted in neutrinos, summing over all components, and another reference energy.

The samples may be used to check how the parameter exploration performs.

In [ ]:
plt.hist(result["samples"]["etot0"], bins=np.logspace(48, 53, 51))
plt.xscale("log")
plt.xlabel("Total energy emitted in neutrinos")
plt.ylabel("Number of posterior samples")

The most relevant is to use these samples to derive constraints on the parameters of interest or eventually the preferred values. Several utility functions are available in `momenta.stats.constraints` for that.

In [ ]:
# get upper limits
limits = momenta.stats.constraints.get_limits(result, CL=0.90)
print(f"Upper limit at 90% credible level on the flux normalisation: {limits['fluxnorm0']:.2g} GeV⁻¹ cm⁻²")
print(f"Upper limit at 90% credible level on the total energy in neutrinos: {limits['etot']:.2g} erg")

# Going further

- `full_example.ipynb` provides an example on how several detectors may be combined to derive better constraints than individual ones.
- `stacking_example.ipynb` walks you through the possibility to perform a stacking analysis where we consider several sources (and several sets of observations) together to constrain some common parameters.
- `tabulated_flux.ipynb` provides a different way to define fluxes. Rather than using well-defined analytical fluxes (e.g., power law), one may use flux functions tabulated from more complex models.

Another interesting feature is the possibility to compute differential limits (per bins of neutrino energy) instead of a single integrated value. This allows appreciating the relative contirbutions of different samples when performing a combination.

In [ ]:
# inputs are similar to the main command, simply adding energy bins
bins_energy = np.logspace(0, 6, 7)
difflimits = momenta.stats.constraints.compute_differential_limits(det, src, parameters, bins_energy)
plt.stairs(difflimits, bins_energy)
plt.xlabel("Energy [GeV]")
plt.ylabel(r"Differential limits [GeV$^{-1}$ cm$^{-2}$]")
plt.xscale("log")
plt.yscale("log")

Another usable outcome of the analysis is the evidence `result["logz"]`, `z` is the integral of the posterior distribution over the full parameter space. It may be used for model comparison, e.g., when computing Bayes factors: $B_{12} = z_1 / z_2$. 

In [ ]:
print(f"Evidence = {result['logz']:.2f} ± {result['logzerr']:.2f}")

Some preliminary implementation is available in `momenta.stats.bayes_factor`, especially to compute the Bayes factor between the background-only and the backgroud+signal hypotheses. 

However, as the prior on the signal parameters S is usually not proper (i.e., $\int \pi(S) dS \neq 1$, such as a flat prior with no well-motivated support), the evidence for the second hypothesis is usually only defined up to a constant, making the computation of Bayes factor tricky.

Some attempt has been made to try to mitigate this, but it should be taken as "work-in-progress".